# Singapore (Completed)

In [ ]:
# COMBINED: Eventbrite + Ticketmaster + Megatix (Featured)
# NO DEDUP — KEEP ALL EVENTS
# pip install tabulate

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
import pandas as pd
import time
import os

driver = webdriver.Chrome()
driver.set_window_size(1400, 1000)

all_rows = []

# -------------------- 1) EVENTBRITE (with auto-scroll + load more) --------------------
try:
    url = 'https://www.eventbrite.sg/d/singapore--singapore/singapore/?subcategories=3006%2C3025&page=1'
    driver.get(url)

    LIST_X = "//*[@id='root']/div/div[2]/div/div/div/div[1]/div/main/div/div/div/section[1]/div/section/div/div/section/ul"
    CARD_X = LIST_X + "/li"
    BASE   = ".//div/div/div[2]/section/div/section[2]/div"

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, LIST_X)))
    time.sleep(1)

    prev = -1
    stable = 0
    for _ in range(80):
        cards = driver.find_elements(By.XPATH, CARD_X)
        count = len(cards)
        if count == prev:
            stable += 1
            if stable >= 2:
                try:
                    load_more = driver.find_element(
                        By.XPATH,
                        "//*[self::button or self::a][contains(.,'Load more') or contains(.,'Show more')]"
                    )
                    driver.execute_script("arguments[0].click();", load_more)
                    time.sleep(1.2)
                    stable = 0
                    prev = -1
                    continue
                except:
                    break
        else:
            stable = 0
        prev = count
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)

    for ev in driver.find_elements(By.XPATH, CARD_X):
        try: title = ev.find_element(By.XPATH, f"{BASE}/a/h3").text.strip()
        except: title = "No Title"
        try: date = ev.find_element(By.XPATH, f"{BASE}/p[1]").text.strip()
        except: date = "No Date"
        try: location = ev.find_element(By.XPATH, f"{BASE}/p[2]").text.strip()
        except: location = "No Location"
        try: link = ev.find_element(By.XPATH, f"{BASE}/a").get_attribute("href")
        except: link = "No Link"
        all_rows.append({"Title": title, "Date": date, "Location": location, "Link": link})

    print(f"🟠 Eventbrite: grabbed {len(driver.find_elements(By.XPATH, CARD_X))} events")
except Exception as e:
    print(f"⚠ Eventbrite error: {e}")

# -------------------- 2) TICKETMASTER (scroll) --------------------
try:
    url = 'https://ticketmaster.sg/categories/concerts'
    driver.get(url)

    EVENTS_X = "//*[@id='events']"
    ANCHOR_X = "//*[@id='events']//a[contains(@href,'/activity/detail/')]"
    TITLE_REL = ".//div[2]/div[2]"
    DATE_REL  = ".//div[2]/div[1]"

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, EVENTS_X)))
    time.sleep(1)

    prev = -1
    for _ in range(60):
        anchors = driver.find_elements(By.XPATH, ANCHOR_X)
        if len(anchors) == prev:
            break
        prev = len(anchors)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)

    for a in driver.find_elements(By.XPATH, ANCHOR_X):
        try: title = a.find_element(By.XPATH, TITLE_REL).text.strip()
        except: title = "No Title"
        try: date = a.find_element(By.XPATH, DATE_REL).text.strip()
        except: date = "No Date"
        link = a.get_attribute("href") or "No Link"
        location = "Singapore"
        all_rows.append({"Title": title, "Date": date, "Location": location, "Link": link})

    print(f"🔵 Ticketmaster: grabbed {len(driver.find_elements(By.XPATH, ANCHOR_X))} events")
except Exception as e:
    print(f"⚠ Ticketmaster error: {e}")

# -------------------- 3) MEGATIX (Featured scroll) --------------------
try:
    url = "https://megatix.com.sg/?page=2"
    driver.get(url)

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__nuxt']")))
    FEATURED_WRAP_X = "//*[@id='__nuxt']/div/div[3]/div/main/section/section[2]/div[2]/div"
    FEATURED_CARD_X = "//*[@id='__nuxt']/div/div[3]/div/main/section/section[2]/div[2]//article[.//h3]"

    wrap = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, FEATURED_WRAP_X)))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", wrap)
    time.sleep(0.8)

    prev = -1
    stable = 0
    for _ in range(60):
        cards = driver.find_elements(By.XPATH, FEATURED_CARD_X)
        count = len(cards)
        if count == prev:
            stable += 1
            if stable >= 2:
                break
        else:
            stable = 0
        prev = count
        driver.execute_script("window.scrollBy(0, 900);")
        time.sleep(1.2)

    for ev in driver.find_elements(By.XPATH, FEATURED_CARD_X):
        try: title = ev.find_element(By.XPATH, ".//h3/span").text.strip()
        except: title = "No Title"
        try: date = ev.find_element(By.XPATH, ".//div[1]/div[1]/span").text.strip()
        except: date = "No Date"
        try: location = ev.find_element(By.XPATH, ".//div[1]/div[3]/span").text.strip()
        except: location = "Singapore"
        try: link = ev.find_element(By.XPATH, ".//ancestor::a").get_attribute("href") or "No Link"
        except: link = "No Link"
        all_rows.append({"Title": title, "Date": date, "Location": location, "Link": link})

    print(f"🟢 Megatix (Featured): grabbed {len(driver.find_elements(By.XPATH, FEATURED_CARD_X))} events")
except Exception as e:
    print(f"⚠ Megatix error: {e}")

driver.quit()

# -------------------- FINAL OUTPUT (append-only, write to Google Sheets) --------------------
# pip install gspread google-auth gspread-dataframe pandas
import json
# time, pandas already imported above

SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1gjYJ1TNwJB8VvEHVNVm7aLVfNIIP0-OrFGymtyvNaAc/edit"
TAB_NAME = "incompleted"
SERVICE_ACCOUNT_JSON = "service_account.json"  # or use env var GOOGLE_SERVICE_ACCOUNT_JSON

# Add scrape timestamp
scraped_at = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
for row in all_rows:
    row["scraped_at_utc"] = scraped_at

df_all = pd.DataFrame(
    all_rows,
    columns=["Title", "Date", "Location", "Link", "scraped_at_utc"]
).reset_index(drop=True)

# Clean up text fields
df_all["Link"] = df_all["Link"].fillna("").str.strip()
df_all["Title"] = df_all["Title"].fillna("").str.strip()
df_all["Date"] = df_all["Date"].fillna("").str.strip()
df_all["Location"] = df_all["Location"].fillna("").str.strip()

# ---- Google Sheets append-only (dedup by Link across runs) ----
import gspread
from google.oauth2.service_account import Credentials

def _authorize():
    scopes = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]
    if "GOOGLE_SERVICE_ACCOUNT_JSON" in os.environ:
        keyfile_dict = json.loads(os.environ["GOOGLE_SERVICE_ACCOUNT_JSON"])
        creds = Credentials.from_service_account_info(keyfile_dict, scopes=scopes)
    else:
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_JSON, scopes=scopes)
    return gspread.authorize(creds)

def _open_or_create_ws(client, url, tab_name):
    sh = client.open_by_url(url)
    try:
        ws = sh.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        ws = sh.add_worksheet(title=tab_name, rows=100, cols=10)
    # Ensure header
    header = ["Title", "Date", "Location", "Link", "scraped_at_utc"]
    current = ws.get_values("A1:E1")
    if not current or current[0] != header:
        ws.update("A1", [header])
    return ws

gc = _authorize()
ws = _open_or_create_ws(gc, SPREADSHEET_URL, TAB_NAME)

# Existing Links (col D) to dedup across runs
existing_links = set(v.strip() for v in ws.col_values(4)[1:] if v and v.strip())

new_rows_df = df_all[~df_all["Link"].astype(str).str.strip().isin(existing_links)].copy()

# Counts/prints aligned with your original console output
print(f"\n✅ Newly scraped this run: {len(df_all)}")
print(f"➕ New (not seen before): {len(new_rows_df)}")

# Append new rows (if any)
if not new_rows_df.empty:
    values = new_rows_df[["Title", "Date", "Location", "Link", "scraped_at_utc"]].values.tolist()
    ws.append_rows(values, value_input_option="RAW")

# Total rows after save (header excluded)
total_after = len(ws.col_values(1)) - 1  # header excluded
print(f"📦 Total in master after save: {total_after}")

print(f"\n💾 Google Sheet updated: {SPREADSHEET_URL} (tab: {TAB_NAME})")



NameError: name 'all_rows' is not defined

: 

# Kuala Lumpur

In [6]:
# --- Live Nation Malaysia: All Events Scraper (deduplicated) ---
# URL: https://www.livenation.my/event/allevents
# Output CSV: livenation_my_all_events.csv
# Columns: Title, Date, Location, Link
# pip install selenium tabulate pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from tabulate import tabulate
import pandas as pd
import time
import os
import re

# ---------------- CONFIG ----------------
OUTPUT_CSV = "all_events_KL.csv"
URL = "https://www.livenation.my/event/allevents"

# Load existing CSV (if any) to prevent duplicates
if os.path.exists(OUTPUT_CSV):
    try:
        existing_df = pd.read_csv(OUTPUT_CSV)
        seen_links = set(existing_df["Link"].dropna().astype(str))
    except Exception:
        existing_df = pd.DataFrame(columns=["Title", "Date", "Location", "Link"])
        seen_links = set()
else:
    existing_df = pd.DataFrame(columns=["Title", "Date", "Location", "Link"])
    seen_links = set()

# ---------------- DRIVER SETUP ----------------
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")
opts.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=opts)
driver.set_window_size(1400, 1000)
all_rows = []

def accept_cookies(drv):
    for xp in [
        "//button[contains(., 'Accept All')]",
        "//button[contains(., 'Accept all')]",
        "//button[contains(., 'I Accept')]",
        "//button[contains(., 'Agree')]",
        "//button[contains(., 'OK')]",
        "//*[@id='onetrust-accept-btn-handler']",
    ]:
        try:
            btn = WebDriverWait(drv, 3).until(EC.element_to_be_clickable((By.XPATH, xp)))
            drv.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except TimeoutException:
            continue
        except Exception:
            continue

def clean_text(s: str) -> str:
    # Collapse whitespace/newlines and trim
    return re.sub(r"\s+", " ", (s or "")).strip()

# ---------------- SCRAPER ----------------
try:
    driver.get(URL)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main']")))
    time.sleep(1.0)
    accept_cookies(driver)

    # Each LI that has an event link and the date container
    ALL_CARDS_X = "//*[@id='main']//ul//li[.//a[contains(@href,'/event/')]]"

    # Scroll to load everything (and click Load/Show more if present)
    prev = -1
    stable = 0
    for _ in range(80):
        cards = driver.find_elements(By.XPATH, ALL_CARDS_X)
        count = len(cards)
        if count == prev:
            stable += 1
            # Try "Load/Show more" if present
            try:
                more = driver.find_element(By.XPATH, "//button[contains(.,'Load more') or contains(.,'Show more')]")
                if more.is_enabled() and more.is_displayed():
                    driver.execute_script("arguments[0].click();", more)
                    time.sleep(1.2)
                    cards = driver.find_elements(By.XPATH, ALL_CARDS_X)
                    count = len(cards)
                    if count > prev:
                        stable = 0
            except Exception:
                pass
            if stable >= 2:
                break
        else:
            stable = 0
        prev = count
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(1.1)

    cards = driver.find_elements(By.XPATH, ALL_CARDS_X)
    print(f"🟢 Found {len(cards)} event cards")

    for li in cards:
        try:
            # Link (primary dedup key)
            try:
                link = li.find_element(By.XPATH, ".//a[contains(@href,'/event/')]").get_attribute("href") or ""
            except Exception:
                link = ""
            if not link or link in seen_links:
                continue

            # Title (your path first)
            title = ""
            for xp in [
                ".//span/div/div/p[1]",   # close to //*[@id='1616419']/span/div/div/p[1]
                ".//p[1]",
                ".//div//p[1]"
            ]:
                try:
                    title = clean_text(li.find_element(By.XPATH, xp).text)
                    if title:
                        break
                except Exception:
                    continue
            if not title:
                title = "No Title"

            # Location (your path first)
            location = ""
            for xp in [
                ".//span/div/div/p[3]/span",  # close to //*[@id='1616419']/span/div/div/p[3]/span
                ".//p[3]/span",
                ".//p[3]"
            ]:
                try:
                    location = clean_text(li.find_element(By.XPATH, xp).text)
                    if location:
                        break
                except Exception:
                    continue
            if not location:
                location = "Malaysia"

            # Date — USE YOUR DIV CONTAINER XPATH RELATIVELY, then fall back to <time>
            # Provided absolute: //*[@id="main"]/div[2]/div[1]/div/div/div/div[2]/ul[1]/li[1]/div/div[1]
            # Relative to each li, this corresponds to: .//div/div[1]
            date_text = ""
            for xp in [
                ".//div/div[1]",          # your container (first choice)
                ".//time",                # fallback: visible text
            ]:
                try:
                    node = li.find_element(By.XPATH, xp)
                    # prefer node.text, otherwise datetime attribute if it's <time>
                    date_text = clean_text(node.text or node.get_attribute("datetime") or "")
                    if date_text:
                        break
                except Exception:
                    continue

            all_rows.append({
                "Title": title,
                "Date": date_text,
                "Location": location,
                "Link": link
            })
            seen_links.add(link)

        except StaleElementReferenceException:
            continue
        except Exception as e:
            print(f"⚠ Skipped one card due to error: {e}")
            continue

    print(f"✅ Live Nation MY (All Events): grabbed {len(all_rows)} new events this run")

except TimeoutException:
    print("⚠ Timeout waiting for Live Nation MY page")
except Exception as e:
    print(f"⚠ Error: {e}")
finally:
    driver.quit()

# ---------------- OUTPUT ----------------
if all_rows:
    new_df = pd.DataFrame(all_rows, columns=["Title", "Date", "Location", "Link"])
    final_df = pd.concat([existing_df, new_df], ignore_index=True)
else:
    final_df = existing_df.copy()

# Final safety dedup
if not final_df.empty:
    final_df["Link"] = final_df["Link"].astype(str).str.strip()
    final_df = final_df.drop_duplicates(subset=["Link"], keep="first")
    final_df = final_df.drop_duplicates(subset=["Title", "Date", "Location"], keep="first")

print(f"\n✅ TOTAL Unique Events: {len(final_df)}")
print(tabulate(final_df.tail(20), headers="keys", tablefmt="github", showindex=False))

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"\n💾 CSV saved: {OUTPUT_CSV}")






🟢 Found 8 event cards
✅ Live Nation MY (All Events): grabbed 2 new events this run

✅ TOTAL Unique Events: 85
| Title                                                          | Date                  | Location                                 | Link                                                                                                                    |   Event Name |   Scraped At (UTC) | TimeScraped      |
|----------------------------------------------------------------|-----------------------|------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|--------------|--------------------|------------------|
| MOZART’S 270TH BIRTHDAY!                                       | DATE 27 Jan 2026      | LOCATION Dewan Filharmonik Petronas KLCC | https://myticket.asia/events/mozarts-270th-birthday/?occurrence=2026-01-27                                              |          nan

In [8]:
"""
BookMyShow MY — Kuala Lumpur scraper (visible tabs; NaN-safe merge; 4-column CSV)
- Stable scrape with slower listing reading
- Safe merge that handles NaN, floats, blanks gracefully
- CSV schema: Title | Date | Location | Link
"""

import os
import time
import sys
import traceback
import hashlib
from typing import Optional, List, Dict

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
    WebDriverException,
)

# ===================== CONFIG =====================
LIST_URL   = "https://my.bookmyshow.com/en/collections/e?cities=Kuala+Lumpur___Kuala+Lumpur"
OUTPUT_CSV = "all_events_KL.csv"

HEADLESS          = False
OPEN_IN_NEW_TAB   = True
PAUSE_ON_EACH_TAB = 0.8
TAKE_ERROR_SHOTS  = True

SLOW_MODE               = True
LISTING_INITIAL_PAUSE   = 1.2
PER_CARD_SCROLL_SETTLE  = 0.35
PER_CARD_READ_RETRIES   = 3
PER_CARD_READ_SLEEP     = 0.25
BETWEEN_CARD_PAUSE      = 0.1

WAIT_SEC          = 20
PAGE_LOAD_TIMEOUT = 25

X_LIST_CONTAINER = "/html/body/div[1]/main/div/div/div[2]"
X_CARD_ANCHOR    = X_LIST_CONTAINER + "/a[{i}]"
X_NAME           = "/html/body/div[1]/main/div/div/div[2]/a[{i}]/div/div[2]/div/div/div[1]"
X_DATE           = "/html/body/div[1]/main/div/div/div[2]/a[{i}]/div/div[2]/div/div/div[2]/span"
X_VENUE_DETAIL   = "/html/body/div[1]/main/div/div/div[2]/div[1]/div[3]/div[4]/div[1]"

# ===================== DRIVER =====================
def build_driver(headless: bool = HEADLESS) -> webdriver.Chrome:
    ua = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
          "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
    opts = webdriver.ChromeOptions()
    opts.add_argument(f"user-agent={ua}")
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,1000")
    opts.add_argument("--disable-notifications")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.page_load_strategy = "eager"

    driver = webdriver.Chrome(options=opts)
    driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
    driver.implicitly_wait(2)
    return driver

# ===================== SAFE STRING HELPERS =====================
def _s(x) -> str:
    """Convert any NaN, None, or float to safe string."""
    if x is None:
        return ""
    try:
        if isinstance(x, float) and pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def normalize_link(u) -> str:
    u = _s(u).strip().lower()
    return u.rstrip("/")

def stable_event_id(title, date_str, location, link_norm) -> str:
    link_norm = _s(link_norm).strip().lower()
    if link_norm:
        return link_norm
    key = "|".join([
        _s(title).strip().lower(),
        _s(date_str).strip().lower(),
        _s(location).strip().lower(),
    ])
    return hashlib.md5(key.encode("utf-8")).hexdigest()

# ===================== HELPERS =====================
def wait_present(driver, xpath, timeout=WAIT_SEC):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

def scroll_lazy(driver, rounds=10, pause=0.5):
    last_h = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = driver.execute_script("return document.body.scrollHeight;")
        if h == last_h:
            break
        last_h = h

def get_text_abs(driver, xpath) -> str:
    try:
        return driver.find_element(By.XPATH, xpath).text.strip()
    except NoSuchElementException:
        return ""
    except StaleElementReferenceException:
        try:
            return driver.find_element(By.XPATH, xpath).text.strip()
        except Exception:
            return ""

def read_text_nonempty(driver, xpath: str, retries: int, sleep_s: float) -> str:
    txt = get_text_abs(driver, xpath)
    if txt:
        return txt
    for _ in range(retries):
        time.sleep(sleep_s)
        txt = get_text_abs(driver, xpath)
        if txt:
            break
    return txt

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def safe_screenshot(driver, path: str):
    try:
        driver.save_screenshot(path)
    except Exception:
        pass

def robust_get(driver, url: str, tries: int = 2, settle: float = 0.4):
    last_err: Optional[Exception] = None
    for _ in range(tries):
        try:
            driver.get(url)
            time.sleep(settle)
            return
        except Exception as e:
            last_err = e
            time.sleep(0.7)
    if last_err:
        raise last_err

def open_in_new_tab(driver, url: str):
    current = driver.current_window_handle
    driver.execute_script("window.open(arguments[0], '_blank');", url)
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
    new_handle = [h for h in driver.window_handles if h != current][-1]
    driver.switch_to.window(new_handle)
    return current, new_handle

def close_tab_and_return(driver, original_handle: str):
    try:
        driver.close()
    finally:
        driver.switch_to.window(original_handle)

# ===================== MERGE (NaN-SAFE) =====================
def merge_with_existing(df_new: pd.DataFrame, path: str) -> pd.DataFrame:
    """
    Merge with old CSV (if exists):
    - Keeps old rows (no removals)
    - No duplicates
    - Overwrites same events with latest scrape
    - NaN-safe
    """
    df_new = df_new.copy()
    for col in ["Title","Date","Location","Link"]:
        if col not in df_new.columns:
            df_new[col] = ""
    df_new[["Title","Date","Location","Link"]] = df_new[["Title","Date","Location","Link"]].fillna("")
    df_new["LinkNorm"] = df_new["Link"].apply(normalize_link)
    df_new["EventID"]  = df_new.apply(
        lambda r: stable_event_id(r.get("Title",""), r.get("Date",""),
                                  r.get("Location",""), r.get("LinkNorm","")), axis=1
    )

    try:
        df_old = pd.read_csv(path)
    except FileNotFoundError:
        return df_new[["Title","Date","Location","Link"]]

    df_old = df_old.copy()
    for col in ["Title","Date","Location","Link"]:
        if col not in df_old.columns:
            df_old[col] = ""
    df_old[["Title","Date","Location","Link"]] = df_old[["Title","Date","Location","Link"]].fillna("")
    df_old["LinkNorm"] = df_old["Link"].apply(normalize_link)
    df_old["EventID"]  = df_old.apply(
        lambda r: stable_event_id(r.get("Title",""), r.get("Date",""),
                                  r.get("Location",""), r.get("LinkNorm","")), axis=1
    )

    merged = pd.concat([df_old, df_new], ignore_index=True)
    merged = merged.drop_duplicates(subset=["EventID"], keep="last")
    merged = merged.drop_duplicates(subset=["LinkNorm"], keep="last")
    return merged[["Title","Date","Location","Link"]]

# ===================== CORE SCRAPE =====================
def collect_listing_cards(driver) -> List[Dict[str, str]]:
    print(f"Opening listing: {LIST_URL}")
    robust_get(driver, LIST_URL, tries=3, settle=0.6)
    wait_present(driver, X_LIST_CONTAINER, timeout=WAIT_SEC)
    scroll_lazy(driver, rounds=10, pause=0.5)
    if SLOW_MODE:
        time.sleep(LISTING_INITIAL_PAUSE)

    anchors = driver.find_elements(By.XPATH, X_LIST_CONTAINER + "/a")
    n = len(anchors)
    print(f"Found {n} cards in listing.")
    cards = []
    for i in range(1, n + 1):
        name_x = X_NAME.format(i=i)
        date_x = X_DATE.format(i=i)
        a_x    = X_CARD_ANCHOR.format(i=i)

        try:
            a_el = driver.find_element(By.XPATH, a_x)
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", a_el)
        except Exception:
            pass
        time.sleep(PER_CARD_SCROLL_SETTLE)

        title = read_text_nonempty(driver, name_x, PER_CARD_READ_RETRIES, PER_CARD_READ_SLEEP)
        date  = read_text_nonempty(driver, date_x, PER_CARD_READ_RETRIES, PER_CARD_READ_SLEEP)

        href = ""
        try:
            a_el = driver.find_element(By.XPATH, a_x)
            href = (a_el.get_attribute("href") or "").strip()
        except Exception:
            pass

        cards.append({"i": i, "title": title, "date": date, "href": href})
        time.sleep(BETWEEN_CARD_PAUSE)
    return cards

def scrape_detail_for_location(driver, url: str, idx: int) -> str:
    if not url:
        return ""
    try:
        if OPEN_IN_NEW_TAB:
            print(f"  → Opening tab #{idx}: {url}")
            original = driver.current_window_handle
            driver.execute_script("window.open(arguments[0], '_blank');", url)
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_handle = [h for h in driver.window_handles if h != original][-1]
            driver.switch_to.window(new_handle)
        else:
            original = driver.current_window_handle
            driver.get(url)

        time.sleep(PAUSE_ON_EACH_TAB)

        try:
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.XPATH, X_VENUE_DETAIL))
            )
            location = driver.find_element(By.XPATH, X_VENUE_DETAIL).text.strip()
            print(f"    ✓ Location scraped: {location[:60]}")
        except TimeoutException:
            location = ""
            print("    ⚠ Location not found (leaving blank).")

        if OPEN_IN_NEW_TAB:
            driver.close()
            driver.switch_to.window(original)
        return location
    except Exception as e:
        print(f"    ✗ Error on detail page: {type(e).__name__}: {e}")
        if TAKE_ERROR_SHOTS:
            ensure_dir("debug_screens")
            safe_screenshot(driver, f"debug_screens/error_event_{idx}.png")
        try:
            if OPEN_IN_NEW_TAB:
                driver.close()
                driver.switch_to.window(original)
        except Exception:
            pass
        return ""

def scrape_bms_kl(driver: webdriver.Chrome) -> pd.DataFrame:
    cards = collect_listing_cards(driver)
    rows = []
    for idx, c in enumerate(cards, start=1):
        title = c["title"]
        date  = c["date"]
        href  = c["href"]
        print(f"\n[{idx}/{len(cards)}] {title} | {date}")
        location = scrape_detail_for_location(driver, href, idx) if href else ""
        rows.append({"Title": title, "Date": date, "Location": location, "Link": href})
    return pd.DataFrame(rows, columns=["Title","Date","Location","Link"])

# ===================== MAIN =====================
def main(headless=HEADLESS):
    driver = build_driver(headless=headless)
    try:
        df_new = scrape_bms_kl(driver)
    finally:
        try:
            driver.quit()
        except Exception:
            pass
    if df_new.empty:
        print("No events found.")
        return
    out = merge_with_existing(df_new, OUTPUT_CSV)
    out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
    print(f"\nSaved {len(out)} total rows -> {OUTPUT_CSV}")

if __name__ == "__main__":
    try:
        main(headless=HEADLESS)
    except Exception as e:
        print(f"[fatal] {type(e).__name__}: {e}")
        traceback.print_exc()
        sys.exit(1)






Opening listing: https://my.bookmyshow.com/en/collections/e?cities=Kuala+Lumpur___Kuala+Lumpur
Found 20 cards in listing.

[1/20] 3P《鑫世纪》巡回演唱会 - 马来西亚站 | 31 Oct 2025
  → Opening tab #1: https://my.bookmyshow.com/en/events/3P-2025/BMS3PGMY
    ✓ Location scraped: Axiata Arena Bukit Jalil

[2/20] Janice Vidal OUT OF FRAME World Tour in Malaysia | 1 Nov 2025
  → Opening tab #2: https://my.bookmyshow.com/en/events/janice-vidal-out-of-frame-world-tour-in-malaysia/BMSJAN25
    ✓ Location scraped: Arena of Stars, Resorts World Genting

[3/20] [CANCELLED] 2025 LOCO CONCERT: SCRAPS TOUR in Asia - Malaysia | 2 Nov 2025
  → Opening tab #3: https://my.bookmyshow.com/en/events/cancelled-2025-loco-concert-scraps-tour-in-asia-malaysia/LOCOMY25
    ✓ Location scraped: Zepp Kuala Lumpur

[4/20] 《佳乐之夜金曲演唱会》马来西亚站 | 2 Nov 2025
  → Opening tab #4: https://my.bookmyshow.com/en/events/jia-le-hokkien-hits-2025/BMSJIALE
    ✓ Location scraped: Axiata Arena Bukit Jalil

[5/20] Engelbert Humperdinck's LAST WALTZ 

In [12]:
# scrape_myticket_asia.py
# pip install selenium pandas tabulate

from __future__ import annotations
import os, time, re
from datetime import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
)

HOME = "https://myticket.asia/"
OUTPUT_CSV = "all_events_KL.csv"
WAIT = 25

# ---------- driver ----------
def make_driver() -> webdriver.Chrome:
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-notifications")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    # Uncomment for headless:
    # opts.add_argument("--headless=new")
    return webdriver.Chrome(options=opts)

# ---------- utils ----------
def slow_scroll_full_page(driver: webdriver.Chrome, step_px=500, nap=0.25, max_loops=200):
    last = -1
    for _ in range(max_loops):
        driver.execute_script("window.scrollBy(0, arguments[0]);", step_px)
        time.sleep(nap)
        y = driver.execute_script("return window.scrollY;")
        if y == last:
            break
        last = y

def click_all_load_more(driver: webdriver.Chrome, max_clicks=50):
    """
    Click all visible 'Load More' variants used by Modern Events Calendar.
    Non-exact: CSS + text-normalization fallback.
    """
    def visible(e): 
        try: return e.is_displayed() and e.is_enabled()
        except: return False

    clicks = 0
    while clicks < max_clicks:
        # Try common MEC 'load more' controls
        buttons = []
        buttons += driver.find_elements(By.CSS_SELECTOR, "a.mec-load-more, button.mec-load-more")
        buttons += driver.find_elements(By.CSS_SELECTOR, ".mec-load-more a, .mec-load-more button")
        # Text fallback (case-insensitive contains 'load more')
        buttons += [e for e in driver.find_elements(By.TAG_NAME, "a") if re.search(r'load\s*more', (e.text or '').lower())]

        # Dedup DOM elements
        seen_ids = set()
        uniq = []
        for b in buttons:
            try:
                key = b.id
                if key not in seen_ids:
                    seen_ids.add(key)
                    uniq.append(b)
            except Exception:
                continue

        clicked_any = False
        for b in uniq:
            if not visible(b): 
                continue
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", b)
                time.sleep(0.25)
                b.click()
                clicked_any = True
                clicks += 1
                time.sleep(1.2)  # allow new cards to render
            except (ElementClickInterceptedException, StaleElementReferenceException):
                continue
            except Exception:
                continue
        if not clicked_any:
            break

def load_existing(path: str) -> pd.DataFrame:
    if os.path.exists(path):
        try:
            return pd.read_csv(path)
        except Exception:
            pass
    return pd.DataFrame(columns=["Title", "Date", "Location", "Link", "TimeScraped"])

def save_append_dedup(df_new: pd.DataFrame, path: str):
    old = load_existing(path)
    all_df = pd.concat([old, df_new], ignore_index=True)
    # primary dedup by Link; secondary by (Title, Date, Location)
    all_df = all_df.drop_duplicates(subset=["Link"], keep="first")
    all_df = all_df.drop_duplicates(subset=["Title", "Date", "Location"], keep="first")
    all_df.to_csv(path, index=False)

def clean_text(s: str) -> str:
    return " ".join((s or "").split())

# ---------- flexible field extraction on detail pages (no exact XPaths) ----------
def extract_title(driver: webdriver.Chrome) -> str:
    # Prefer page H1 within main area, fallback to any h1
    for css in ["main h1", "#main-content h1", "h1"]:
        els = driver.find_elements(By.CSS_SELECTOR, css)
        for el in els:
            t = clean_text(el.text)
            if t: return t
    return ""

def extract_date(driver: webdriver.Chrome) -> str:
    # Try <time>, then common date classes, then definition list dd near date-like labels
    # 1) time tags
    for el in driver.find_elements(By.TAG_NAME, "time"):
        t = clean_text(el.text)
        if t: return t
    # 2) common date-ish classes/labels
    for css in ["[class*='date']", "[class*='time']"]:
        for el in driver.find_elements(By.CSS_SELECTOR, css):
            t = clean_text(el.text)
            if t and len(t) > 2: return t
    # 3) simple dl/dd extraction: pick the dd whose preceding label mentions date/time
    dds = driver.find_elements(By.CSS_SELECTOR, "dl dd")
    dts = driver.find_elements(By.CSS_SELECTOR, "dl dt")
    for i, dt in enumerate(dts):
        label = (dt.text or "").lower()
        if any(k in label for k in ["date", "time", "when"]):
            try:
                t = clean_text(dds[i].text)
                if t: return t
            except Exception:
                pass
    # fallback: first dd with non-empty text
    for el in dds:
        t = clean_text(el.text)
        if t: return t
    return ""

def extract_location(driver: webdriver.Chrome) -> str:
    # Look for 'venue' or 'location' labels first
    dds = driver.find_elements(By.CSS_SELECTOR, "dl dd")
    dts = driver.find_elements(By.CSS_SELECTOR, "dl dt")
    for i, dt in enumerate(dts):
        label = (dt.text or "").lower()
        if any(k in label for k in ["venue", "location", "where"]):
            try:
                t = clean_text(dds[i].text)
                if t: return t
            except Exception:
                pass
    # Generic class-based fallbacks
    for css in ["[class*='venue']", "[class*='location']"]:
        for el in driver.find_elements(By.CSS_SELECTOR, css):
            t = clean_text(el.text)
            if t and len(t) > 2: return t
    # last resort: look for address-like blocks
    for tag in ["address", "p", "span", "div"]:
        for el in driver.find_elements(By.TAG_NAME, tag):
            txt = clean_text(el.text)
            if any(k in txt.lower() for k in ["hall", "theatre", "theater", "ballroom", "arena", "convention", "stadium", "klcc"]):
                if len(txt) <= 120:
                    return txt
    return ""

# ---------- main ----------
def main():
    driver = make_driver()
    wait = WebDriverWait(driver, WAIT)
    rows = []

    try:
        driver.get(HOME)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1.0)

        # Load everything we can see: scroll + load more + scroll again
        slow_scroll_full_page(driver)
        click_all_load_more(driver)
        slow_scroll_full_page(driver)

        # Collect ALL event links (no exact XPath):
        # MEC usually wraps titles in h4 > a inside containers with id starting mec_skin_events_.
        card_links = driver.find_elements(By.CSS_SELECTOR, "div[id^='mec_skin_events_'] h4 a, section[id^='mec_skin_events_'] h4 a")
        # If homepage changes, also try generic h4 > a with myticket links
        if not card_links:
            card_links = [a for a in driver.find_elements(By.CSS_SELECTOR, "h4 a") if "myticket.asia" in (a.get_attribute("href") or "")]

        # Dedup by href
        links = []
        seen = set()
        for el in card_links:
            try:
                href = (el.get_attribute("href") or "").strip()
                title_from_list = clean_text(el.text)
                if href and href not in seen:
                    seen.add(href)
                    links.append((title_from_list, href))
            except StaleElementReferenceException:
                continue

        if not links:
            print("No event links found. Site structure may have changed.")
            return

        # Visit each link in a new tab and extract fields
        for title_from_list, href in links:
            driver.execute_script("window.open(arguments[0], '_blank');", href)
            driver.switch_to.window(driver.window_handles[-1])

            try:
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            except TimeoutException:
                pass

            name = extract_title(driver) or title_from_list
            date = extract_date(driver)
            location = extract_location(driver)

            row = {
                "Title": clean_text(name),
                "Date": clean_text(date),
                "Location": clean_text(location),
                "Link": href,
                "TimeScraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            }
            rows.append(row)

            # close tab and return
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(0.4)

        df = pd.DataFrame(rows)

        # -------- NEW: show how many new events were added --------
        old_df = load_existing(OUTPUT_CSV)
        old_count = len(old_df)

        save_append_dedup(df, OUTPUT_CSV)

        updated_df = load_existing(OUTPUT_CSV)
        new_total = len(updated_df)
        added_count = new_total - old_count
        print(f"\n🆕 {added_count} new event(s) added ({new_total} total now in file).")
        # -------- END NEW --------

        from tabulate import tabulate
        print(tabulate(df, headers="keys", tablefmt="github", showindex=False))
        print(f"\nSaved {len(df)} unique rows → {OUTPUT_CSV}")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()







🆕 0 new event(s) added (85 total now in file).
| Title                                                                 | Date                  | Location                                    | Link                                                                                                                    | TimeScraped         |
|-----------------------------------------------------------------------|-----------------------|---------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|---------------------|
| THE INCREDIBLE VOYAGE OF ALASDAIR MALLOY                              | DATE 23 May 2026      | LOCATION Dewan Filharmonik Petronas KLCC    | https://myticket.asia/events/the-incredible-voyage-of-alasdair-malloy/?occurrence=2026-05-23                            | 2025-10-28 17:19:58 |
| A TRIBUTE TO SHARIFAH AINI                                            | DATE 06 Jun 20

# Jarkata

In [14]:
# --- Megatix Indonesia: Featured Events Scraper (deduplicated) ---
# Output CSV: all_jakarta.csv
# Works for https://megatix.co.id/
# pip install selenium tabulate pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tabulate import tabulate
import pandas as pd
import time
import os
from datetime import datetime   # <-- added for timestamp

# ---------------- CONFIG ----------------
OUTPUT_CSV = "all_jakarta.csv"
URL = "https://megatix.co.id/"

# Load existing CSV (if any) to prevent duplicates
if os.path.exists(OUTPUT_CSV):
    existing_df = pd.read_csv(OUTPUT_CSV)
    seen_links = set(existing_df["Link"].dropna())
else:
    existing_df = pd.DataFrame()
    seen_links = set()

# ---------------- DRIVER SETUP ----------------
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=opts)
driver.set_window_size(1400, 1000)
all_rows = []

# ---------------- SCRAPER ----------------
try:
    driver.get(URL)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__nuxt']")))
    time.sleep(1.0)

    FEATURED_CARD_X = "//*[@id='__nuxt']/div/div[3]/div/main/section/section[2]/div[2]//article[.//h3]"

    # Scroll to load all cards
    prev = -1
    stable = 0
    for _ in range(60):
        cards = driver.find_elements(By.XPATH, FEATURED_CARD_X)
        count = len(cards)
        if count == prev:
            stable += 1
            if stable >= 2:
                break
        else:
            stable = 0
        prev = count
        driver.execute_script("window.scrollBy(0, 900);")
        time.sleep(1.2)

    cards = driver.find_elements(By.XPATH, FEATURED_CARD_X)
    print(f"🟢 Found {len(cards)} featured cards")

    for ev in cards:
        try:
            title = ev.find_element(By.XPATH, ".//h3/span").text.strip()
        except:
            title = "No Title"
        try:
            date = ev.find_element(By.XPATH, ".//div[1]/div[1]/span").text.strip()
        except:
            date = "No Date"
        try:
            location = ev.find_element(By.XPATH, ".//div[1]/div[3]/span").text.strip()
        except:
            location = "Indonesia"
        try:
            link = ev.find_element(By.XPATH, ".//ancestor::a").get_attribute("href") or "No Link"
        except:
            link = "No Link"

        # Skip duplicates already in CSV
        if link in seen_links:
            continue

        all_rows.append({
            "Title": title,
            "Date": date,
            "Location": location,
            "Link": link,
            "TimeScraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # <-- added line
        })
        seen_links.add(link)

    print(f"✅ Megatix Indonesia (Featured): grabbed {len(all_rows)} new events")

except TimeoutException:
    print("⚠ Timeout waiting for Megatix Indonesia page")
except Exception as e:
    print(f"⚠ Error: {e}")
finally:
    driver.quit()

# ---------------- OUTPUT ----------------
if all_rows:
    new_df = pd.DataFrame(all_rows)
    final_df = pd.concat([existing_df, new_df], ignore_index=True)
else:
    final_df = existing_df

print(f"\n✅ TOTAL Unique Events: {len(final_df)}")
print(tabulate(final_df.tail(20), headers="keys", tablefmt="github", showindex=False))

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"\n💾 CSV saved: {OUTPUT_CSV}")




🟢 Found 7 featured cards
✅ Megatix Indonesia (Featured): grabbed 0 new events

✅ TOTAL Unique Events: 9
| Title                                       | Date             | Location             | Link                                                       |
|---------------------------------------------|------------------|----------------------|------------------------------------------------------------|
| X-Clusive Presents: Bali NYE 2025 with Tyga | Wed, 31 Dec 2025 | The Stage            | https://megatix.co.id/events/nye-2025-with-tyga            |
| X-Clusive Presents: Bali NYE 2025 with Tyga | Wed, 31 Dec 2025 | The Stage            | https://megatix.co.id/events/nye-2025-with-tyga            |
| MEDUZA                                      | Sat, 25 Oct 2025 | Savaya Bali          | https://megatix.co.id/events/meduza-2510                   |
| ILOVEUNITED Jakarta                         | Sat, 25 Oct 2025 | Community Park PIK 2 | https://megatix.co.id/events/iloveunited-jakarta   

# Hong Kong

In [15]:
# --- HONG KONG: Live Nation only ---
# Output CSV: all_hong_kong.csv
# pip install selenium tabulate pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from tabulate import tabulate
import pandas as pd
import time, os, unicodedata, shutil
from urllib.parse import urlparse
from datetime import datetime  # <-- added for timestamp

# =================== CONFIG ===================
OUTPUT_CSV = "all_hong_kong.csv"
BACKUP_CSV = OUTPUT_CSV + ".bak"
LIVE_HOME = "https://www.livenation.hk/en"
WAIT = 25

# =================== DRIVER ===================
ua = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
      "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")
opts = webdriver.ChromeOptions()
opts.add_argument(f"user-agent={ua}")
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option("useAutomationExtension", False)
opts.page_load_strategy = "eager"

driver = webdriver.Chrome(options=opts)
driver.set_window_size(1400, 1000)
driver.set_page_load_timeout(60)
driver.set_script_timeout(60)

# =================== UTILS ===================
def accept_cookies(driver):
    for by, sel in [
        (By.ID, "onetrust-accept-btn-handler"),
        (By.XPATH, "//*[@id='onetrust-accept-btn-handler']"),
        (By.XPATH, "//*[self::button or self::a][contains(.,'Accept All') or contains(.,'Accept')]"),
    ]:
        try:
            el = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((by, sel)))
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.2)
            break
        except Exception:
            continue

def safe_text(el):
    try:
        return unicodedata.normalize("NFKC", " ".join(el.text.split()))
    except Exception:
        return ""

def norm(s):
    return " ".join(unicodedata.normalize("NFKC", (s or "").strip().lower()).split())

def canonicalize_link(link):
    if not link:
        return ""
    p = urlparse(link)
    return f"{(p.netloc or '').lower()}{p.path or ''}"

def make_event_id(platform, link, title, date):
    canon = canonicalize_link(link)
    if canon:
        return f"{platform}|{canon}"
    return f"{norm(platform)}|{norm(title)}|{norm(date)}"

# =================== LOAD OLD CSV ===================
seen_ids_persisted = set()
old_df = None
column_order = ["Platform", "Title", "Date", "Location", "Link", "TimeScraped"]  # <-- added column here

if os.path.exists(OUTPUT_CSV):
    try:
        old_df = pd.read_csv(OUTPUT_CSV, dtype=str, keep_default_na=False)
        if list(old_df.columns):
            column_order = list(old_df.columns)
        for _, r in old_df.iterrows():
            seen_ids_persisted.add(
                make_event_id(
                    str(r.get("Platform","")),
                    str(r.get("Link","")),
                    str(r.get("Title","")),
                    str(r.get("Date","")),
                )
            )
        print(f"↺ Loaded {len(seen_ids_persisted)} previously-scraped rows")
    except Exception as e:
        print(f"⚠ Could not read existing CSV: {e}")

all_rows = []
seen_links_this_run = set()

def append_row(platform, title, date_text, location, link):
    row = {
        "Platform": platform,
        "Title": title or "No Title",
        "Date": date_text or "No Date",
        "Location": location or "Hong Kong",
        "Link": link or "No Link",
        "TimeScraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # <-- added timestamp
    }
    eid = make_event_id(row["Platform"], row["Link"], row["Title"], row["Date"])
    if eid in seen_ids_persisted:
        return
    canon = canonicalize_link(row["Link"])
    if canon and canon in seen_links_this_run:
        return
    if canon:
        seen_links_this_run.add(canon)
    all_rows.append(row)

# =================== LIVE NATION HK ===================
def find_all_events_section():
    try:
        return WebDriverWait(driver, 12).until(
            EC.presence_of_element_located((By.XPATH, "//*[self::h2 or self::h3][contains(.,'All Events')]/ancestor::section[1]"))
        )
    except TimeoutException:
        return WebDriverWait(driver, 12).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='main']/div/div[6]/section/div/div/div[2]"))
        )

def scroll_deep(times=8, pause=0.45):
    for _ in range(times):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(pause)

def scrape_livenation_page(label="LiveNationHK"):
    section = find_all_events_section()
    cards = section.find_elements(By.XPATH, ".//li[.//a[@href]]")
    if not cards:
        cards = section.find_elements(By.XPATH, ".//ul[1]/li | .//div[contains(@class,'Card')]")
    for li in cards:
        link, title, location, date_text = "", "", "Hong Kong", ""
        try:
            a = li.find_element(By.XPATH, ".//a[@href]")
            link = a.get_attribute("href") or ""
        except NoSuchElementException:
            pass
        for xp in [".//p[contains(@class,'title')][1]", ".//p[1]", ".//*[self::h3 or self::h2 or self::p][1]"]:
            try:
                title = safe_text(li.find_element(By.XPATH, xp))
                if title: break
            except NoSuchElementException:
                continue
        for xp in [".//p[contains(@class,'location')]", ".//p[2]"]:
            try:
                l = safe_text(li.find_element(By.XPATH, xp))
                if l: location = l; break
            except NoSuchElementException:
                continue
        for xp in [".//time", ".//p[contains(@class,'date')][1]", ".//p[2]"]:
            try:
                d = safe_text(li.find_element(By.XPATH, xp))
                if d and d.lower() != location.lower():
                    date_text = d; break
            except NoSuchElementException:
                continue
        append_row(label, title, date_text, location, link)

def run_livenation():
    driver.get(LIVE_HOME)
    accept_cookies(driver)
    scroll_deep(8, 0.45)
    scrape_livenation_page("LiveNationHK")
    # Try "Page 2"
    try:
        nxt = WebDriverWait(driver, 6).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='main']/div/div[6]//nav//li[3]/button"))
        )
        driver.execute_script("arguments[0].click();", nxt)
        time.sleep(1.0)
        scroll_deep(8, 0.45)
        scrape_livenation_page("LiveNationHK")
    except TimeoutException:
        pass

# =================== RUN ===================
try:
    run_livenation()
finally:
    try:
        driver.quit()
    except Exception:
        pass

# =================== SAVE (append-only) ===================
new_df = pd.DataFrame(all_rows, dtype=str).reset_index(drop=True)

if os.path.exists(OUTPUT_CSV) and old_df is not None:
    if not new_df.empty:
        new_df["__id"] = new_df.apply(
            lambda r: make_event_id(r.get("Platform",""), r.get("Link",""), r.get("Title",""), r.get("Date","")),
            axis=1
        )
        new_df = new_df[~new_df["__id"].isin(seen_ids_persisted)].drop(columns="__id")
    for col in column_order:
        if col not in new_df.columns:
            new_df[col] = ""
    new_df = new_df[column_order]
    final_df = pd.concat([old_df, new_df], ignore_index=True)
else:
    for col in column_order:
        if col not in new_df.columns:
            new_df[col] = ""
    new_df = new_df[column_order]
    final_df = new_df

# Fill missing TimeScraped for older rows
if "TimeScraped" in final_df.columns:
    final_df["TimeScraped"] = final_df["TimeScraped"].replace("", pd.NA).fillna(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

added = len(new_df)
print(f"\n✅ New rows appended this run: {added}")
print(f"📦 Total rows in file: {len(final_df)}")
print(tabulate(final_df.tail(max(1, min(added, 20))), headers="keys", tablefmt="github", showindex=False))

tmp = OUTPUT_CSV + ".tmp"
final_df.to_csv(tmp, index=False, encoding="utf-8-sig")
if os.path.exists(OUTPUT_CSV):
    try:
        shutil.copy2(OUTPUT_CSV, BACKUP_CSV)
    except Exception:
        pass
os.replace(tmp, OUTPUT_CSV)
print(f"\n💾 CSV updated: {OUTPUT_CSV}")
if os.path.exists(BACKUP_CSV):
    print(f"🧯 Backup saved: {BACKUP_CSV}")







↺ Loaded 20 previously-scraped rows

✅ New rows appended this run: 2
📦 Total rows in file: 25
| Platform     | Title                                                | Date       | Location   | Link                                                                                                               |
|--------------|------------------------------------------------------|------------|------------|--------------------------------------------------------------------------------------------------------------------|
| LiveNationHK | DAY6 10th Anniversary Tour <The DECADE> in HONG KONG | SUN 18 JAN | DAY6       | https://www.livenation.hk/en/event/day6-10th-anniversary-tour-the-decade-in-hong-kong-hong-kong-tickets-edp1630128 |
| LiveNationHK | Blue 25th Anniversary Tour                           | SAT 07 FEB | Blue       | https://www.livenation.hk/en/event/blue-25th-anniversary-tour-hong-kong-tickets-edp1630645                         |

💾 CSV updated: all_hong_kong.csv
🧯 Backup sav

In [18]:
# --- HKTicketing Scraper (XPath-only; no fallbacks) ---
# Site: https://premier.hkticketing.com/
# Output CSV: all_hong_kong.csv (Platform, Title, Date, Location, Link)
# pip install selenium tabulate pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from tabulate import tabulate
import pandas as pd
import time, os, math, sys

HOME = "https://premier.hkticketing.com/"
OUTPUT_CSV = "all_hong_kong.csv"

# Tunables
WAIT = 25                 # explicit wait seconds
PAGELOAD_TIMEOUT = 60     # page load timeout seconds
SCRIPT_TIMEOUT = 60
NAV_RETRIES = 4           # number of times to retry loading homepage

# Load previous results (for de-dup by Link)
if os.path.exists(OUTPUT_CSV):
    try:
        existing_df = pd.read_csv(OUTPUT_CSV)
        if "Link" in existing_df.columns:
            seen_links = set(existing_df["Link"].dropna().astype(str))
        else:
            seen_links = set()
    except Exception:
        existing_df = pd.DataFrame(columns=["Platform", "Title", "Date", "Location", "Link"])
        seen_links = set()
else:
    existing_df = pd.DataFrame(columns=["Platform", "Title", "Date", "Location", "Link"])
    seen_links = set()

# ------------- Driver -------------
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")
opts.add_argument("--disable-popup-blocking")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option("useAutomationExtension", False)
# If you want headless, uncomment:
# opts.add_argument("--headless=new")
# opts.add_argument("--window-size=1440,1000")

driver = webdriver.Chrome(options=opts)
driver.set_window_size(1400, 1000)
driver.set_page_load_timeout(PAGELOAD_TIMEOUT)
driver.set_script_timeout(SCRIPT_TIMEOUT)
actions = ActionChains(driver)

def save_debug(prefix: str):
    try:
        png = f"{prefix}_screenshot.png"
        html = f"{prefix}_source.html"
        driver.save_screenshot(png)
        with open(html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"⚠ Saved debug artifacts: {png}, {html}")
    except Exception:
        pass

def js_ready() -> bool:
    try:
        return driver.execute_script("return document.readyState") == "complete"
    except Exception:
        return False

def wait_for_home_ready(timeout=WAIT):
    """Wait until the homepage is fully ready:
       - document.readyState == 'complete'
       - and one of the key sections exists
    """
    end = time.time() + timeout
    while time.time() < end:
        if js_ready():
            try:
                # Either 'More Events' or 'Hero' nav visible is sufficient
                if driver.find_elements(By.XPATH, '//*[@id="moreEventsSection"]') or \
                   driver.find_elements(By.XPATH, '//*[@id="heroNavPaging"]'):
                    return True
            except Exception:
                pass
        time.sleep(0.3)
    return False

def get_with_retries(url: str, attempts=NAV_RETRIES) -> None:
    """Robust navigation with retries + backoff."""
    last_err = None
    for i in range(1, attempts + 1):
        try:
            driver.get(url)
            # Tiny scroll to nudge lazy content
            driver.execute_script("window.scrollTo(0, 50);")
            time.sleep(0.3)
            if wait_for_home_ready(timeout=WAIT + 5):
                return
            else:
                raise TimeoutException("Home not ready within wait window.")
        except Exception as e:
            last_err = e
            save_debug(f"load_attempt_{i}")
            # Backoff
            sleep_s = min(2 * i, 8)
            print(f"⚠ Load attempt {i}/{attempts} failed: {e}. Retrying in {sleep_s}s...")
            time.sleep(sleep_s)
    # If we get here, all attempts failed
    raise TimeoutException(f"Failed to load {url} after {attempts} attempts: {last_err}")

def safe_get_text(by, xp, timeout=WAIT):
    el = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, xp)))
    return el.text.strip()

def safe_click(by, xp, timeout=WAIT):
    el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, xp)))
    # bring into view + small pause to ensure it’s interactable
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.15)
    driver.execute_script("arguments[0].click();", el)

rows = []

try:
    # Open homepage robustly
    get_with_retries(HOME, attempts=NAV_RETRIES)

    # -----------------------------
    # (1) MORE EVENTS section loop
    # -----------------------------
    idx = 1
    while True:
        tile_xp = f'//*[@id="moreEventsSection"]/div[{idx}]/div[2]/a'
        title_click_xp = f'//*[@id="moreEventsSection"]/div[{idx}]/div[2]/a/div[2]/strong'

        try:
            tile = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.XPATH, tile_xp)))
        except TimeoutException:
            # No more tiles at this index → end loop
            break

        link = tile.get_attribute("href") or ""

        # Click the <strong> element (as requested)
        safe_click(By.XPATH, title_click_xp)
        # allow any dynamic panels to render
        time.sleep(0.6)

        # Venue & Date (strict XPaths)
        venue_xp = '//*[@id="ctl00_ctl00_uiBodyMain_uiBodyRight_uiPerfSelector_uiPerfSelectorUpdatePanel"]/div[2]/div[3]/p[1]'
        date_xp  = '//*[@id="ctl00_ctl00_uiBodyMain_uiBodyRight_uiPerfSelector_uiPerfSelectorUpdatePanel"]/div[2]/div[3]/p[2]'

        try:
            venue = safe_get_text(By.XPATH, venue_xp, timeout=WAIT)
        except Exception:
            venue = ""

        try:
            date_time = safe_get_text(By.XPATH, date_xp, timeout=WAIT)
        except Exception:
            date_time = ""

        # Title: use page <title> (no alternative selectors)
        try:
            page_title = driver.title.strip()
        except Exception:
            page_title = ""
        title_val = page_title if page_title else f"Event {idx}"

        if not link:
            try:
                link = driver.current_url
            except Exception:
                link = ""

        if link and link not in seen_links:
            rows.append({
                "Platform": "HKTicketing",
                "Title": title_val,
                "Date": date_time,      # map DateTime -> Date (no parsing, keep as-is)
                "Location": venue,      # map Venue -> Location
                "Link": link
            })
            seen_links.add(link)

        # Back to homepage and re-wait for readiness (prevents half-rendered state)
        driver.back()
        if not wait_for_home_ready(timeout=WAIT + 5):
            # If coming back lands on a stale state, reload home robustly
            print("ℹ Re-loading home after back navigation...")
            get_with_retries(HOME, attempts=2)
        time.sleep(0.4)

        idx += 1

    # -----------------------------
    # (2) HERO flow (exact steps)
    # -----------------------------
    safe_click(By.XPATH, '//*[@id="heroNavPaging"]/a[1]')
    time.sleep(0.5)

    hero_name = safe_get_text(By.XPATH, '//*[@id="heroModuleInner"]/div[2]/div[2]/div[1]/h2/a')

    safe_click(By.XPATH, '//*[@id="heroModuleInner"]/div[3]/div[2]/div[2]/a')
    # wait for the app container to render its details
    WebDriverWait(driver, WAIT).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-scroll-container"]'))
    )
    time.sleep(0.6)

    hero_venue = safe_get_text(By.XPATH, '//*[@id="app-scroll-container"]/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/p[5]')
    hero_datetime = safe_get_text(By.XPATH, '//*[@id="app-scroll-container"]/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/p[6]')

    try:
        hero_link = driver.current_url
    except Exception:
        hero_link = ""

    if hero_link and hero_link not in seen_links:
        rows.append({
            "Platform": "HKTicketing",
            "Title": hero_name,
            "Date": hero_datetime,
            "Location": hero_venue,
            "Link": hero_link
        })
        seen_links.add(hero_link)

except TimeoutException as e:
    print(f"⚠ Timeout while loading or scraping: {e}")
    save_debug("fatal_timeout")
except NoSuchElementException as e:
    print(f"⚠ Missing element via provided XPath: {e}")
    save_debug("fatal_no_such_element")
except Exception as e:
    print(f"⚠ Unexpected error: {e}")
    save_debug("fatal_error")
finally:
    try:
        driver.quit()
    except Exception:
        pass

# ---------------- OUTPUT ----------------
if rows:
    new_df = pd.DataFrame(rows, columns=["Platform", "Title", "Date", "Location", "Link"])
    # Align previous dataframe to new schema if needed
    missing_cols = [c for c in ["Platform", "Title", "Date", "Location", "Link"] if c not in existing_df.columns]
    if missing_cols:
        for c in missing_cols:
            existing_df[c] = ""  # fill missing
        existing_df = existing_df[["Platform", "Title", "Date", "Location", "Link"]]
    final_df = pd.concat([existing_df, new_df], ignore_index=True)
else:
    # Align schema even if no new rows
    if set(existing_df.columns) != set(["Platform", "Title", "Date", "Location", "Link"]):
        aligned = pd.DataFrame(columns=["Platform", "Title", "Date", "Location", "Link"])
        for c in ["Platform", "Title", "Date", "Location", "Link"]:
            if c in existing_df.columns:
                aligned[c] = existing_df[c]
            else:
                aligned[c] = ""
        final_df = aligned
    else:
        final_df = existing_df.copy()

if not final_df.empty:
    final_df["Link"] = final_df["Link"].astype(str).str.strip()
    final_df = final_df.drop_duplicates(subset=["Link"], keep="first")

# ------- NEW: print if new events were added and list them -------
try:
    existing_links_before = set(existing_df["Link"].astype(str).str.strip())
except Exception:
    existing_links_before = set()

if rows:
    # new_df contains only rows scraped this run; detect which are actually new vs. pre-existing
    new_mask = ~new_df["Link"].astype(str).str.strip().isin(existing_links_before)
    actually_new = new_df[new_mask].copy()
    added_count = len(actually_new)
    if added_count > 0:
        print(f"\n🆕 Added {added_count} new event(s):")
        for _, r in actually_new.iterrows():
            print(f"  • {r['Title']} | {r['Date']} | {r['Location']} | {r['Link']}")
    else:
        print("\nℹ No new events added (everything scraped already existed).")
else:
    print("\nℹ No new events scraped this run.")
# ------- END NEW -------

print(f"\n✅ TOTAL Unique Events: {len(final_df)}")
print(tabulate(final_df.tail(20), headers="keys", tablefmt="github", showindex=False))

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"\n💾 CSV saved: {OUTPUT_CSV}")





⚠ Timeout while loading or scraping: Message: 

⚠ Saved debug artifacts: fatal_timeout_screenshot.png, fatal_timeout_source.html

ℹ No new events scraped this run.

✅ TOTAL Unique Events: 25
| Platform     | Title                                                                            | Date                                                       | Location                                                                            | Link                                                                                                                                                            |
|--------------|----------------------------------------------------------------------------------|------------------------------------------------------------|-------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------

# Jarkata 

In [19]:
# --- Megatix Indonesia: Featured Events Scraper ---
# Output CSV: all_jakarta.csv
# Works for https://megatix.co.id/
# pip install selenium tabulate pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tabulate import tabulate
import pandas as pd
import time
import os  # <-- added to read existing CSV for "new events" comparison

# ---------------- CONFIG ----------------
OUTPUT_CSV = "all_jakarta.csv"
URL = "https://megatix.co.id/"

driver = webdriver.Chrome()
driver.set_window_size(1400, 1000)
all_rows = []

# ---------------- SCRAPER ----------------
try:
    driver.get(URL)

    # Wait for base element
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__nuxt']")))
    time.sleep(1.0)

    FEATURED_WRAP_X = "//*[@id='__nuxt']/div/div[3]/div/main/section/section[2]/div[2]/div"
    FEATURED_CARD_X = "//*[@id='__nuxt']/div/div[3]/div/main/section/section[2]/div[2]//article[.//h3]"

    # Scroll and load all featured cards
    prev = -1
    stable = 0
    for _ in range(60):
        cards = driver.find_elements(By.XPATH, FEATURED_CARD_X)
        count = len(cards)
        if count == prev:
            stable += 1
            if stable >= 2:
                break
        else:
            stable = 0
        prev = count
        driver.execute_script("window.scrollBy(0, 900);")
        time.sleep(1.2)

    print(f"🟢 Found {len(driver.find_elements(By.XPATH, FEATURED_CARD_X))} featured cards")

    for ev in driver.find_elements(By.XPATH, FEATURED_CARD_X):
        try:
            title = ev.find_element(By.XPATH, ".//h3/span").text.strip()
        except:
            title = "No Title"
        try:
            date = ev.find_element(By.XPATH, ".//div[1]/div[1]/span").text.strip()
        except:
            date = "No Date"
        try:
            location = ev.find_element(By.XPATH, ".//div[1]/div[3]/span").text.strip()
        except:
            location = "Indonesia"
        try:
            link = ev.find_element(By.XPATH, ".//ancestor::a").get_attribute("href") or "No Link"
        except:
            link = "No Link"

        all_rows.append({
            "Title": title,
            "Date": date,
            "Location": location,
            "Link": link
        })

    print(f"✅ Megatix Indonesia (Featured): grabbed {len(all_rows)} events")

except TimeoutException:
    print("⚠ Timeout waiting for Megatix Indonesia page")
except Exception as e:
    print(f"⚠ Error: {e}")

finally:
    driver.quit()

# ---------------- OUTPUT ----------------
df = pd.DataFrame(all_rows).reset_index(drop=True)

# --- NEW: Compare with existing CSV and print any newly added events ---
if os.path.exists(OUTPUT_CSV):
    try:
        old_df = pd.read_csv(OUTPUT_CSV)
        old_links = set(old_df.get("Link", pd.Series(dtype=str)).astype(str).str.strip())
    except Exception:
        old_links = set()
else:
    old_links = set()

# Normalize links for comparison and detect new ones
if not df.empty:
    df["Link"] = df["Link"].astype(str).str.strip()
    new_mask = ~df["Link"].isin(old_links)
    new_events = df[new_mask].copy()
    if len(new_events) > 0:
        print(f"\n🆕 {len(new_events)} new event(s) since last run:")
        for _, r in new_events.iterrows():
            print(f"  • {r['Title']} | {r['Date']} | {r['Location']} | {r['Link']}")
    else:
        print("\nℹ No new events added since last run.")
else:
    print("\nℹ No events scraped this run (empty listing).")
# --- END NEW ---

print(f"\n✅ TOTAL Raw Events: {len(df)}")
print(tabulate(df, headers="keys", tablefmt="github", showindex=False))

df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"\n💾 CSV saved: {OUTPUT_CSV}")


🟢 Found 7 featured cards
✅ Megatix Indonesia (Featured): grabbed 7 events

ℹ No new events added since last run.

✅ TOTAL Raw Events: 7
| Title                                       | Date             | Location          | Link                                                       |
|---------------------------------------------|------------------|-------------------|------------------------------------------------------------|
| X-Clusive Presents: Bali NYE 2025 with Tyga | Wed, 31 Dec 2025 | The Stage         | https://megatix.co.id/events/nye-2025-with-tyga            |
| X-Clusive Presents: Bali NYE 2025 with Tyga | Wed, 31 Dec 2025 | The Stage         | https://megatix.co.id/events/nye-2025-with-tyga            |
| HAYDEN JAMES                                | Sat, 08 Nov 2025 | Savaya Bali       | https://megatix.co.id/events/hayden-james-0811             |
| DEFECTED                                    | Sun, 09 Nov 2025 | Savaya Bali       | https://megatix.co.id/events/defected

# Bangkok

In [20]:
# thai ticket major
# pip install selenium pandas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from datetime import datetime
import pandas as pd
import time

HOME = "https://www.thaiticketmajor.com/concert/"

# ---- Your detail-page XPaths (kept as-is) ----
NAME_X  = "/html/body/div[1]/main/div[1]/div[2]/div/div[2]/div/div[1]/div/h1/font/font"
VENUE_X = "/html/body/div[1]/main/div[1]/div[2]/div/div[2]/div/div[2]/div[1]/ul/li[2]/p/span/font/font"
PRICE_X = "/html/body/div[1]/main/div[1]/div[2]/div/div[2]/div/div[2]/div[2]/ul/li[2]/div"

# Listing: we’ll collect all anchors in section[2] (your click path lives under here)
# This is more reliable than clicking the nested /font/font node.
LIST_ANCHORS_X = "//body/div[1]/main/section[2]//a[@href]"

# ================== APPEND+DEDUP ADD-ON (no changes to your code below) ==================
import atexit, os, re

_OUTPUT_CSV_PATH = "all_bangkok.csv"

def _normalize_link(u: str) -> str:
    u = (u or "").strip().lower()
    u = re.sub(r"#.*$", "", u)           # drop fragments
    u = re.sub(r"\?.*$", "", u)          # drop query params
    return u.rstrip("/")

try:
    _old_df_ttm = pd.read_csv(_OUTPUT_CSV_PATH)
except Exception:
    _old_df_ttm = pd.DataFrame()

def _merge_back_ttm():
    """Runs AFTER your script's own to_csv.
       - Restores previous rows
       - Removes duplicates
       - Prints which events are newly added this run (or none)
    """
    try:
        new_df = pd.read_csv(_OUTPUT_CSV_PATH)
    except Exception:
        # If the fresh write failed, nothing to merge
        return

    # Ensure consistent columns even if earlier files differ
    need_cols = ["Event Name", "Venue", "Ticket Prices", "Link", "Scraped At (UTC)"]
    for df in (_old_df_ttm, new_df):
        for c in need_cols:
            if c not in df.columns:
                df[c] = ""

    # Normalize links for strong dedup
    _old_df_ttm["LinkNorm"] = _old_df_ttm["Link"].map(_normalize_link)
    new_df["LinkNorm"]      = new_df["Link"].map(_normalize_link)

    # --- Identify which rows are NEW vs the old file (before merging) ---
    old_keys = set(_old_df_ttm["LinkNorm"].astype(str))
    # For rows with empty LinkNorm, fallback to (Event Name, Venue)
    if "" in old_keys:
        old_fallback = set(zip(
            _old_df_ttm["Event Name"].astype(str).str.strip().str.lower(),
            _old_df_ttm["Venue"].astype(str).str.strip().str.lower()
        ))
    else:
        old_fallback = set()

    newly_added_rows = []
    for _, r in new_df.iterrows():
        ln = str(r.get("LinkNorm", "") or "")
        if ln:
            if ln not in old_keys:
                newly_added_rows.append(r)
        else:
            key = (
                str(r.get("Event Name","")).strip().lower(),
                str(r.get("Venue","")).strip().lower()
            )
            if key not in old_fallback:
                newly_added_rows.append(r)

    # Merge and dedup
    merged = pd.concat([_old_df_ttm, new_df], ignore_index=True)

    # Primary dedup by normalized link (keep first = preserve earliest row)
    merged = merged.drop_duplicates(subset=["LinkNorm"], keep="first")

    # Secondary guard when links are missing/unstable
    merged = merged.drop_duplicates(subset=["Event Name", "Venue"], keep="first")

    # Persist without helper column
    merged = merged.drop(columns=["LinkNorm"], errors="ignore")
    merged.to_csv(_OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")

    # --- Print summary of new events ---
    if newly_added_rows:
        print(f"\n🆕 {len(newly_added_rows)} new event(s) added this run:")
        for r in newly_added_rows:
            title = str(r.get("Event Name","")).strip()
            venue = str(r.get("Venue","")).strip()
            link  = str(r.get("Link","")).strip()
            date  = str(r.get("Scraped At (UTC)","")).strip()
            print(f"  • {title} | {venue} | {link} | scraped {date}")
    else:
        print("\nℹ No new events added this run.")

# Ensure merge runs after your script finishes writing the CSV
atexit.register(_merge_back_ttm)
# ================== END ADD-ON ==================

def build_driver():
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-notifications")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.page_load_strategy = "eager"
    d = webdriver.Chrome(options=opts)
    d.set_page_load_timeout(90)
    return d

def wait(drv, xp, sec=25, clickable=False):
    cond = EC.element_to_be_clickable if clickable else EC.presence_of_element_located
    return WebDriverWait(drv, sec).until(cond((By.XPATH, xp)))

def t(drv, xp):
    try:
        el = drv.find_element(By.XPATH, xp)
        # textContent is safer with nested <font> etc.
        return " ".join((el.get_attribute("textContent") or "").split())
    except:
        return ""

def main():
    driver = build_driver()
    rows = []
    try:
        driver.get(HOME)
        # Wait for listing anchors to exist
        wait(driver, LIST_ANCHORS_X)
        time.sleep(0.5)

        anchors = driver.find_elements(By.XPATH, LIST_ANCHORS_X)
        hrefs = []
        for a in anchors:
            href = a.get_attribute("href")
            # Keep only event-detail-like links (most are /event/... or /concert/...)
            if href and "thaiticketmajor.com" in href and href not in hrefs:
                hrefs.append(href)

        if not hrefs:
            print("No events found on the listing.")
            return

        for i, href in enumerate(hrefs, 1):
            try:
                driver.get(href)
                # Wait for name node to appear
                wait(driver, NAME_X, sec=20)

                name  = t(driver, NAME_X)
                venue = t(driver, VENUE_X)
                price = t(driver, PRICE_X)

                rows.append({
                    "Event Name": name,
                    "Venue": venue,
                    "Ticket Prices": price,
                    "Link": href,
                    "Scraped At (UTC)": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
                })

                # If you strictly want to “go back and click the rest”, uncomment these two lines:
                # driver.back()
                # wait(driver, LIST_ANCHORS_X)

            except (TimeoutException, StaleElementReferenceException):
                rows.append({
                    "Event Name": "",
                    "Venue": "",
                    "Ticket Prices": "",
                    "Link": href,
                    "Scraped At (UTC)": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
                })
                # Try to return to list if something failed
                try:
                    driver.get(HOME)
                    wait(driver, LIST_ANCHORS_X)
                except TimeoutException:
                    pass

        pd.DataFrame(rows).to_csv("all_bangkok.csv", index=False, encoding="utf-8-sig")
        print(f"Saved {len(rows)} rows to all_bangkok.csv")

    finally:
        try: driver.quit()
        except: pass

if __name__ == "__main__":
    main()







KeyboardInterrupt: 

In [21]:
# ticketmelon (all asia)



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from datetime import datetime
import pandas as pd
import time

URL = "https://www.livenationtero.co.th/en?utm_source=chatgpt.com"

LIST_X   = "//*[@id='upcoming-shows']/div/ul"
ITEM_X   = LIST_X + "/li[{i}]"
ANCHOR_X = ITEM_X + "/a"
NAME_X   = ITEM_X + "/a/p[1]"      # event name
DATE_X   = ITEM_X + "/a/small"     # event date  ✅

VENUE_X_PRIMARY = "//*[@id='main']/div/div[1]/div/div[2]/div/p[4]"
VENUE_FALLBACKS = [
    "//*[@id='main']//p[contains(., 'Venue')]/following-sibling::p[1]",
    "//*[@id='main']//div[contains(@class,'event') or contains(@class,'details')]//p[contains(@class,'venue')]",
    "//*[contains(@class,'EventDetails')]//p[contains(@class,'venue')]",
]

# ================== APPEND + DEDUP ADD-ON (ONLY ADDITIONS BELOW) ==================
import os, re, atexit

_OUTPUT_CSV_PATH = "all_bangkok.csv"

def _normalize_link(u: str) -> str:
    u = (u or "").strip().lower()
    u = re.sub(r"#.*$", "", u)          # drop fragments
    u = re.sub(r"\?.*$", "", u)         # drop query params
    return u.rstrip("/")

# Preload old CSV (if any) so we can merge it back after your own to_csv runs
try:
    _old_df_lnt = pd.read_csv(_OUTPUT_CSV_PATH)
except Exception:
    _old_df_lnt = pd.DataFrame()

def _merge_back_lnt():
    """Runs AFTER your script writes all_bangkok.csv.
       Merges old + new, removes duplicates, saves back to file,
       and prints which events are newly added (or none).
    """
    try:
        new_df = pd.read_csv(_OUTPUT_CSV_PATH)
    except Exception:
        return  # nothing written, nothing to merge

    # Ensure the expected schema exists on both sides
    need_cols = ["Event Name", "Event Date", "Venue", "Link", "Scraped At (UTC)"]
    for df in (_old_df_lnt, new_df):
        for c in need_cols:
            if c not in df.columns:
                df[c] = ""

    # Normalize links for robust dedup (handles tracking params, trailing slashes)
    _old_df_lnt["LinkNorm"] = _old_df_lnt["Link"].map(_normalize_link)
    new_df["LinkNorm"]      = new_df["Link"].map(_normalize_link)

    # ---- Determine which rows are NEW compared to old file ----
    old_linknorms = set(_old_df_lnt["LinkNorm"].astype(str))
    # Fallback identity if Link is blank/unstable: (Event Name, Venue)
    old_fallback_keys = set(zip(
        _old_df_lnt["Event Name"].astype(str).str.strip().str.lower(),
        _old_df_lnt["Venue"].astype(str).str.strip().str.lower()
    ))

    newly_added_rows = []
    for _, r in new_df.iterrows():
        ln = str(r.get("LinkNorm", "") or "")
        if ln:
            if ln not in old_linknorms:
                newly_added_rows.append(r)
        else:
            key = (
                str(r.get("Event Name","")).strip().lower(),
                str(r.get("Venue","")).strip().lower()
            )
            if key not in old_fallback_keys:
                newly_added_rows.append(r)

    # ---- Merge & deduplicate globally ----
    merged = pd.concat([_old_df_lnt, new_df], ignore_index=True)
    merged = merged.drop_duplicates(subset=["LinkNorm"], keep="first")
    merged = merged.drop_duplicates(subset=["Event Name", "Venue"], keep="first")

    # Persist without helper column
    merged = merged.drop(columns=["LinkNorm"], errors="ignore")
    merged.to_csv(_OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")

    # ---- Print summary of new events (or none) ----
    if newly_added_rows:
        print(f"\n🆕 {len(newly_added_rows)} new event(s) added this run:")
        for r in newly_added_rows:
            title = str(r.get("Event Name","")).strip()
            date  = str(r.get("Event Date","")).strip()
            venue = str(r.get("Venue","")).strip()
            link  = str(r.get("Link","")).strip()
            when  = str(r.get("Scraped At (UTC)","")).strip()
            print(f"  • {title} | {date} | {venue} | {link} | scraped {when}")
    else:
        print("\nℹ No new events added this run.")

# Register the merge so it runs automatically at program exit
atexit.register(_merge_back_lnt)
# ================== END ADD-ON ==================

def build_driver():
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-notifications")
    return webdriver.Chrome(options=opts)

def wait_xpath(drv, xpath, sec=25):
    return WebDriverWait(drv, sec).until(EC.presence_of_element_located((By.XPATH, xpath)))

def wait_clickable(drv, xpath, sec=25):
    return WebDriverWait(drv, sec).until(EC.element_to_be_clickable((By.XPATH, xpath)))

def node_text(drv, xpath):
    try:
        el = drv.find_element(By.XPATH, xpath)
        # textContent catches nested spans / line breaks better than .text sometimes
        txt = el.get_attribute("textContent") or ""
        return " ".join(txt.split())
    except:
        return ""

def get_venue_with_fallbacks(drv, timeout=8):
    end = time.time() + timeout
    try:
        el = WebDriverWait(drv, min(4, timeout)).until(
            EC.presence_of_element_located((By.XPATH, VENUE_X_PRIMARY))
        )
        txt = el.text.strip()
        if txt: return txt
    except TimeoutException:
        pass
    for xp in VENUE_FALLBACKS:
        if time.time() > end: break
        try:
            el = WebDriverWait(drv, max(1, int(end - time.time()))).until(
                EC.presence_of_element_located((By.XPATH, xp))
            )
            txt = el.text.strip()
            if txt: return txt
        except TimeoutException:
            continue
    return ""

def main():
    driver = build_driver()
    rows = []
    try:
        driver.get(URL)
        wait_xpath(driver, LIST_X)
        items = driver.find_elements(By.XPATH, LIST_X + "/li")
        n = len(items)
        if n == 0:
            print("No events found under #upcoming-shows.")
            return

        for i in range(1, n + 1):
            try:
                li = driver.find_element(By.XPATH, ITEM_X.format(i=i))
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", li)
                time.sleep(0.2)

                event_name = node_text(driver, NAME_X.format(i=i))
                event_date = node_text(driver, DATE_X.format(i=i))   # ✅ now using <small>

                a = wait_clickable(driver, ANCHOR_X.format(i=i))
                href = a.get_attribute("href")
                a.click()

                venue = get_venue_with_fallbacks(driver, timeout=8)

                rows.append({
                    "Event Name": event_name,
                    "Event Date": event_date,
                    "Venue": venue,
                    "Link": href or driver.current_url,
                    "Scraped At (UTC)": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
                })

            except (TimeoutException, StaleElementReferenceException):
                rows.append({
                    "Event Name": event_name if 'event_name' in locals() else "",
                    "Event Date": event_date if 'event_date' in locals() else "",
                    "Venue": "",
                    "Link": href if 'href' in locals() else "",
                    "Scraped At (UTC)": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
                })
            finally:
                try:
                    driver.back()
                    wait_xpath(driver, LIST_X)
                except TimeoutException:
                    driver.get(URL)
                    wait_xpath(driver, LIST_X)

        if rows:
            pd.DataFrame(rows).to_csv("all_bangkok.csv", index=False, encoding="utf-8-sig")
            print(f"Saved {len(rows)} rows to all_bangkok.csv")
        else:
            print("No rows collected.")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


# need to add code to show how many new events were added




Saved 5 rows to all_bangkok.csv


# Australia

In [ ]:
"""
Ticketmaster AU — Dance/Electronic (date-filtered) scraper
- Source URL (example): https://www.ticketmaster.com.au/browse/dance-electronic-catid-201/music-rid-10001#pageInfo?startDate=2025-10-27&endDate=2025-12-31
- Columns: Title, Date, Location, Link, Source, ScrapedAtUTC

Usage (VS Code / terminal):
    pip install selenium pandas
    python scrape_ticketmaster_au_elec.py
"""

import time
import sys
from datetime import datetime, timezone

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
)

# ===================== CONFIG =====================
START_DATE = "2025-10-27"
END_DATE   = "2025-12-31"

# Your provided (category) page + date filters.
# NOTE: Put the "#pageInfo" anchor before the query string as per your link.
BASE_URL = (
    "https://www.ticketmaster.com.au/browse/dance-electronic-catid-201/music-rid-10001"
    "#pageInfo"
    f"?startDate={START_DATE}&endDate={END_DATE}"
)

OUTPUT_CSV = "all_australia.csv"
WAIT_SEC   = 25

# Your provided XPaths
X_SECTION  = "//*[@id='pageInfo']/div[1]/div/div[2]/div[2]/div[1]"
X_LIST     = X_SECTION + "/div[2]/div[2]/ul"
X_ITEMS    = X_LIST + "/li"

# Relative XPaths from each <li> item (based on your example for li[1])
X_NAME_REL      = ".//div[1]/div/div[2]/div[2]/span[1]/span"
X_LOCATION_REL  = ".//div[1]/div/div[2]/div[2]/span[2]/span[2]"
X_DATE_REL      = ".//div[1]/div/div[2]/div[1]/span/span[2]"
# Link: try to grab an <a> within the card
X_LINK_REL      = ".//a[@href]"

# ===================== DRIVER =====================
def build_driver(headless: bool = True) -> webdriver.Chrome:
    ua = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
          "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
    opts = webdriver.ChromeOptions()
    opts.add_argument(f"user-agent={ua}")
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-notifications")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    # Selenium Manager will fetch the correct driver automatically (Selenium 4.6+).
    return webdriver.Chrome(options=opts)

# ===================== HELPERS =====================
def wait_for(xpath: str, driver: webdriver.Chrome, timeout: int = WAIT_SEC):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

def scroll_lazy(driver: webdriver.Chrome, max_rounds: int = 18, pause: float = 0.75):
    """
    Scroll down repeatedly to trigger lazy loading.
    Stops early if page height no longer grows.
    """
    last_h = 0
    for i in range(max_rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = driver.execute_script("return document.body.scrollHeight;")
        if h == last_h:
            break
        last_h = h

def get_text_safe(node, rel_xpath: str) -> str:
    try:
        return node.find_element(By.XPATH, rel_xpath).text.strip()
    except NoSuchElementException:
        return ""
    except StaleElementReferenceException:
        try:
            return node.find_element(By.XPATH, rel_xpath).text.strip()
        except Exception:
            return ""

def get_link_safe(node, rel_xpath: str) -> str:
    try:
        a = node.find_element(By.XPATH, rel_xpath)
        href = a.get_attribute("href") or ""
        return href.strip()
    except NoSuchElementException:
        return ""
    except StaleElementReferenceException:
        try:
            a = node.find_element(By.XPATH, rel_xpath)
            href = a.get_attribute("href") or ""
            return href.strip()
        except Exception:
            return ""

# ===================== MAIN SCRAPER =====================
def scrape_ticketmaster_au(driver: webdriver.Chrome) -> pd.DataFrame:
    print(f"Opening: {BASE_URL}")
    driver.get(BASE_URL)

    # Wait until the section & list exist
    try:
        wait_for(X_SECTION, driver)
        wait_for(X_LIST, driver)
    except TimeoutException:
        raise TimeoutException("Could not find the events section on the page (XPaths may have changed).")

    # Scroll to load all items
    scroll_lazy(driver, max_rounds=20, pause=0.8)

    # Grab items
    try:
        items = driver.find_elements(By.XPATH, X_ITEMS)
    except NoSuchElementException:
        items = []

    print(f"Found {len(items)} event cards (raw).")

    rows = []
    scraped_at = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
    source = "Ticketmaster AU"

    for idx, li in enumerate(items, start=1):
        try:
            title    = get_text_safe(li, X_NAME_REL)
            location = get_text_safe(li, X_LOCATION_REL)
            date_str = get_text_safe(li, X_DATE_REL)
            link     = get_link_safe(li, X_LINK_REL)

            # Skip empty rows with no title
            if not title:
                continue

            rows.append({
                "Title": title,
                "Date": date_str,
                "Location": location,
                "Link": link,
                "Source": source,
                "ScrapedAtUTC": scraped_at,
            })
        except Exception as e:
            # Keep going if one card explodes
            print(f"[warn] Skipped li #{idx} due to error: {e}")
            continue

    df = pd.DataFrame(rows, columns=["Title", "Date", "Location", "Link", "Source", "ScrapedAtUTC"])
    # Deduplicate by Link, then by (Title, Date, Location)
    if not df.empty:
        before = len(df)
        df = df.drop_duplicates(subset=["Link"]).copy()
        df = df.drop_duplicates(subset=["Title", "Date", "Location"]).copy()
        after = len(df)
        print(f"De-duplicated: {before} -> {after}")

    return df

def merge_with_existing(df_new: pd.DataFrame, path: str) -> pd.DataFrame:
    try:
        df_old = pd.read_csv(path)
        merged = pd.concat([df_old, df_new], ignore_index=True)
        # Global dedupe again
        merged = merged.drop_duplicates(subset=["Link"]).drop_duplicates(subset=["Title", "Date", "Location"])
        return merged
    except FileNotFoundError:
        return df_new

def main():
    driver = build_driver(headless=True)
    try:
        df = scrape_ticketmaster_au(driver)
    finally:
        try:
            driver.quit()
        except Exception:
            pass

    if df.empty:
        print("No events found (empty DataFrame).")
        return

    out = merge_with_existing(df, OUTPUT_CSV)
    out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
    print(f"\nSaved {len(out)} total rows -> {OUTPUT_CSV}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[fatal] {e}")
        sys.exit(1)


# need to add code to show how many new events were added



Opening: https://www.ticketmaster.com.au/browse/dance-electronic-catid-201/music-rid-10001#pageInfo?startDate=2025-10-27&endDate=2025-12-31
Found 20 event cards (raw).
De-duplicated: 20 -> 20

Saved 20 total rows -> all_australia.csv


# Bali